In [1]:
import requests
import json
from pathlib import Path 
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()  # Load environment variables

In [3]:
BASE_URL = "https://api.eka.care"
CLIENT_ID = os.getenv('EKA_CLIENT_ID')
CLIENT_SECRET = os.getenv('EKA_CLIENT_SECRET')

# General functions wrapping APIs

In [13]:
def get_client_token(client_id, client_secret):
    """
    Retrieve JWT token using client credentials.
    """
    url = f"{BASE_URL}/connect-auth/v1/account/login"
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    
    response.raise_for_status()
    return response.json().get("access_token")

def upload_audio(access_token, file_paths, mode, txnid):
    """
    Upload files to the server.
    """
    url = f"{BASE_URL}/voice/upload"
    headers = {
        "auth": access_token,
    }

    querystring = {"mode":f"{mode}","txnid":f"{txnid}"}

     # Prepare files for upload
    files = [
        ("file", (Path(file_path).name, open(file_path, "rb"), "audio/wav"))
        for file_path in file_paths
    ]

    response = requests.post(url, headers=headers, files=files, params=querystring)

    return response.text


def retrieve_scribe_output(access_token, txid):
    """
    Retrieve JSON data using the document ID.
    """
    identifier = str(txid)
    url = f"{BASE_URL}/health/api/v1/fhir/retrieve?identifier={identifier}"
    headers = {"auth": access_token}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

### Authenticate

In [ ]:
# Step 1: Get client token
access_token = get_client_token(CLIENT_ID, CLIENT_SECRET)
print(f"Access Token: {access_token}")

### Upload and scribe audio

In [ ]:
# Upload audio file
audio_path = "../../../assets/eka_example-audio.mp3"
tx_id = "testing-example-audio-dictation"

file_paths = [audio_path]
response = upload_audio(access_token, file_paths, mode='dictation', txnid=tx_id)
print(response)

### Fetch FHIR record

Even though the Results API (above) contains base64 encoded FHIR document, there is another API to just fetch the FHIR record

In [ ]:
# Step 3: Retrieve JSON data
json_data = retrieve_scribe_output(access_token, tx_id)
print("Retrieved JSON Data:")
print(json.dumps(json_data, indent=4))

### Delete Document

This API gives developers to completely erase every trace of the document from EkaCare's servers. 

NOTE: not deleting document incurrs a storage cost

In [ ]:
delete_status = delete_document(access_token, document_id)
print(f"Document deleted successfully. Status code: {delete_status}")